In [1]:
import networkx as nx
import numpy as np
import pandas as pd
from rdflib import Graph, URIRef
from rdflib.extras.external_graph_libs import rdflib_to_networkx_multidigraph
from rdflib.namespace import RDF, RDFS, SOSA
from sklearn.model_selection import KFold
from tqdm import tqdm

from viscars.data import DataLoader
from viscars.evaluation.evaluators import Evaluator
from viscars.evaluation.metrics import MetricType
from viscars.evaluation.metrics.factory import MetricFactory
from viscars.namespace import DASHB
from viscars.recommenders import Recommender
from viscars.utils import visualize_graph

In [2]:
class PPR(Recommender):

    def __init__(self, graph: Graph, verbose=False, alpha=0.8, tol=10e-6):
        super().__init__(graph, verbose)

        self.alpha = alpha
        self.tolerance = tol
        self.personalization = None

    def _build_model(self):
        graph_ = Graph()
        graph_ += self.graph.triples((None, None, None))

        self.items = list(graph_.subjects(RDF.type, SOSA.ObservableProperty))
        self.model = rdflib_to_networkx_multidigraph(graph_).to_undirected()

    def set_personalization(self, weight_uid=0, weight_cid=0):
        weight_others = 1 - weight_uid - weight_cid

        if weight_uid == 0:
            weight_uid = weight_others
        if weight_cid == 0:
            weight_cid = weight_others

        self.personalization = (weight_uid, weight_cid, weight_others)

    def _personalization(self, uid, cid):
        personalization = {}

        for node in self.model.nodes:
            uri = str(node)
            if uri in uid:
                personalization[node] = self.personalization[0]
            elif uri in cid:
                personalization[node] = self.personalization[1]
            else:
                personalization[node] = self.personalization[2]

        return personalization

    def run(self, uid: [] = None, cid: [] = None):
        if uid is not None and cid is not None:  # TODO: Check if UID and CID exist in graph -> ZeroDivisionError
            weights = self._personalization(uid, cid)
            return nx.pagerank(self.model, alpha=self.alpha, personalization=weights,
                               tol=self.tolerance)

        return nx.pagerank(self.model, alpha=self.alpha, tol=self.tolerance)

    def predict(self, uid: [] = None, cid: [] = None, **kwargs):
        pr = self.run(uid, cid)
        
        items_ = []
        qry = f'''
            PREFIX dashb: <http://dynamicdashboard.ilabt.imec.be/broker/ontologies/dashboard#>
            PREFIX sosa: <http://www.w3.org/ns/sosa/>
            PREFIX ssn-ext: <http://dynamicdashboard.ilabt.imec.be/broker/ontologies/ssn-extension/>

            SELECT ?property WHERE {{      
                ?sensor ssn-ext:subSystemOf <{patient}> ;
                    sosa:observes ?property .
            }}
        '''
        items_ = [property_[0] for property_ in self.graph.query(qry)]

        recommendations = \
            [{'contextId': str(cid), 'itemId': str(item), 'score': p} for item, p in pr.items()
             if item in items_]
        return sorted(recommendations, key=lambda n: n['score'], reverse=True)

    def top_n(self, uid: [], cid: [], n: int, **kwargs):
        pass


In [3]:
graph = Graph()
graph.parse(f'./data/protego/graph.ttl', format='n3')
graph.parse(f'./data/protego/protego_zplus.ttl', format='n3')
d_loader = DataLoader(graph)

In [4]:
recommender = PPR(graph)
recommender.set_personalization(0.7, 0.3)

In [5]:
patient = 'http://example.com/tx/patients/zplus_6'
properties = []

qry = f'''
    PREFIX dashb: <http://dynamicdashboard.ilabt.imec.be/broker/ontologies/dashboard#>
    PREFIX sosa: <http://www.w3.org/ns/sosa/>
    PREFIX ssn-ext: <http://dynamicdashboard.ilabt.imec.be/broker/ontologies/ssn-extension/>
    
    SELECT ?property WHERE {{      
        ?sensor ssn-ext:subSystemOf <{patient}> ;
            sosa:observes ?property .
    }}
'''

result = graph.query(qry)
for row in result:
    properties.append(row.property)

In [6]:
recommendations = recommender.predict(['https://dynamicdashboard.ilabt.imec.be/users/10'], ['http://example.com/tx/patients/zplus_6'])

for r in recommendations:
    item = str(r['itemId'])
    score = r['score']
    print(f'[{score}] {item}')

[0.001148031328737956] https://webthing.protego.dynamicdashboard.ilabt.imec.be/things/zplus_6.lifestyle/properties/enriched-call
[0.0010976117838766152] https://webthing.protego.dynamicdashboard.ilabt.imec.be/things/zplus_6.60%253A77%253A71%253A7D%253A93%253AD7%252Fservice0009/properties/org.dyamand.types.health.GlucoseLevel
[0.0008632407731234491] https://webthing.protego.dynamicdashboard.ilabt.imec.be/things/zplus_6.B0%253A91%253A22%253AFB%253AD0%253A78%252Fservice0009/properties/org.dyamand.types.health.GlucoseLevel
[0.0008614956191866942] https://webthing.protego.dynamicdashboard.ilabt.imec.be/things/zplus_6.00%253A1C%253A05%253AFE%253A44%253A6B%252Fservice0029/properties/org.dyamand.types.health.SpO2
[0.0008614619872436425] https://webthing.protego.dynamicdashboard.ilabt.imec.be/things/zplus_6.00%253A1C%253A05%253AFF%253AA9%253A4E%252Fservice0029/properties/org.dyamand.types.health.SpO2
[0.0008602570567442537] https://webthing.protego.dynamicdashboard.ilabt.imec.be/things/zplus_6.

In [7]:
diseases = []

qry = f'''
    PREFIX dashb:<http://dynamicdashboard.ilabt.imec.be/broker/ontologies/dashboard#>
    PREFIX saref4ehaw: <https://saref.etsi.org/saref4ehaw/>
    
    SELECT ?disease WHERE {{
       <{patient}> a saref4ehaw:HealthActor ;
           saref4ehaw:hasChronicDisease ?disease .
    }}
'''

result = graph.query(qry)

for row in result:
    diseases.append(row[0])
    
print(diseases)

[rdflib.term.URIRef('http://example.com/tx/diseases/Diabetes'), rdflib.term.URIRef('http://example.com/tx/diseases/Hypoglycemia')]


## Using PropertyClasses

In [8]:
patient = 'http://example.com/tx/patients/zplus_6'
properties_classes = []

qry = f'''
    PREFIX dashb: <http://dynamicdashboard.ilabt.imec.be/broker/ontologies/dashboard#>
    PREFIX sosa: <http://www.w3.org/ns/sosa/>
    PREFIX ssn-ext: <http://dynamicdashboard.ilabt.imec.be/broker/ontologies/ssn-extension/>
    
    SELECT ?propertyClass WHERE {{      
        ?sensor ssn-ext:subSystemOf <{patient}> ;
            sosa:observes ?property .
        
        ?property a ?propertyClass .
        ?propertyClass a sosa:ObservableProperty .
    }}
'''

result = graph.query(qry)
for row in result:
    properties_classes.append(row.propertyClass)
    
print(properties_classes)

[rdflib.term.URIRef('https://dahcc.idlab.ugent.be/Homelab/SensorsAndWearables/org.dyamand.types.health.BodyTemperature'), rdflib.term.URIRef('https://dahcc.idlab.ugent.be/Homelab/SensorsAndWearables/org.dyamand.types.health.BodyTemperature'), rdflib.term.URIRef('https://dahcc.idlab.ugent.be/Homelab/SensorsAndWearables/org.dyamand.types.health.SpO2'), rdflib.term.URIRef('https://dahcc.idlab.ugent.be/Homelab/SensorsAndWearables/org.dyamand.types.health.SpO2'), rdflib.term.URIRef('https://dahcc.idlab.ugent.be/Homelab/SensorsAndWearables/org.dyamand.types.health.DiastolicBloodPressure'), rdflib.term.URIRef('https://dahcc.idlab.ugent.be/Homelab/SensorsAndWearables/org.dyamand.types.health.GlucoseLevel'), rdflib.term.URIRef('https://dahcc.idlab.ugent.be/Homelab/SensorsAndWearables/org.dyamand.types.health.GlucoseLevel'), rdflib.term.URIRef('https://dahcc.idlab.ugent.be/Homelab/SensorsAndWearables/org.dyamand.types.health.SystolicBloodPressure'), rdflib.term.URIRef('https://dahcc.idlab.ugent.

In [9]:
recommendations = recommender.predict(['https://dynamicdashboard.ilabt.imec.be/users/10'], ['http://example.com/tx/patients/zplus_6'])

for r in recommendations:
    item = str(r['itemId'])
    score = r['score']
    print(f'[{score}] {item}')

[0.001148031328737956] https://webthing.protego.dynamicdashboard.ilabt.imec.be/things/zplus_6.lifestyle/properties/enriched-call
[0.0010976117838766152] https://webthing.protego.dynamicdashboard.ilabt.imec.be/things/zplus_6.60%253A77%253A71%253A7D%253A93%253AD7%252Fservice0009/properties/org.dyamand.types.health.GlucoseLevel
[0.0008632407731234491] https://webthing.protego.dynamicdashboard.ilabt.imec.be/things/zplus_6.B0%253A91%253A22%253AFB%253AD0%253A78%252Fservice0009/properties/org.dyamand.types.health.GlucoseLevel
[0.0008614956191866942] https://webthing.protego.dynamicdashboard.ilabt.imec.be/things/zplus_6.00%253A1C%253A05%253AFE%253A44%253A6B%252Fservice0029/properties/org.dyamand.types.health.SpO2
[0.0008614619872436425] https://webthing.protego.dynamicdashboard.ilabt.imec.be/things/zplus_6.00%253A1C%253A05%253AFF%253AA9%253A4E%252Fservice0029/properties/org.dyamand.types.health.SpO2
[0.0008602570567442537] https://webthing.protego.dynamicdashboard.ilabt.imec.be/things/zplus_6.

# Evaluation

In [10]:
qry = '''
    PREFIX dashb: <http://dynamicdashboard.ilabt.imec.be/broker/ontologies/dashboard#>
    PREFIX sosa: <http://www.w3.org/ns/sosa/>
    PREFIX ssn-ext: <http://dynamicdashboard.ilabt.imec.be/broker/ontologies/ssn-extension/>

    SELECT ?user ?context ?property WHERE {
        ?sensor ssn-ext:subSystemOf ?context ;
            sosa:observes ?property .
            
        ?widget dashb:hasProperty ?property ;
                dashb:createdBy ?user .                
    }
'''

ratings = {'user': [], 'item': [], 'rating': [], 'context': []}
for row in graph.query(qry):
    user_ = row[0]
    context_ = row[1]
    item_ = row[2]
    
    ratings.get('user').append(user_)
    ratings.get('item').append(item_)
    ratings.get('rating').append(5.0)
    ratings.get('context').append(context_)
ratings_df = pd.DataFrame.from_dict(ratings)

In [11]:
# Context metadata
qry = '''
    PREFIX dashb: <http://dynamicdashboard.ilabt.imec.be/broker/ontologies/dashboard#>
    PREFIX sosa: <http://www.w3.org/ns/sosa/>
    PREFIX ssn-ext: <http://dynamicdashboard.ilabt.imec.be/broker/ontologies/ssn-extension/>

    SELECT ?context WHERE {
        ?sensor ssn-ext:subSystemOf ?context ;
            sosa:observes ?property .
    }
'''
context_metadata = {'id': []}

result = graph.query(qry)
for row in result:
    context_metadata.get('id').append(row[0])
context_metadata_df = pd.DataFrame.from_dict(context_metadata)

# User metadata
qry = '''
    PREFIX dashb: <http://dynamicdashboard.ilabt.imec.be/broker/ontologies/dashboard#>

    SELECT ?user ?username ?role WHERE {
        ?user dashb:memberOf ?role .
    }
'''
user_metadata = {'id': [], 'type': []}

result = graph.query(qry)
for row in result:
    user_metadata.get('id').append(row[0])
    user_metadata.get('type').append(row[1])
user_metadata_df = pd.DataFrame.from_dict(user_metadata)

# Item metadata
qry = '''
    PREFIX dashb: <http://dynamicdashboard.ilabt.imec.be/broker/ontologies/dashboard#>
    PREFIX sosa: <http://www.w3.org/ns/sosa/>

    SELECT ?property WHERE {
        ?property a sosa:ObservableProperty .
    }
'''
item_metadata = {'id': []}

result = graph.query(qry)
for row in result:
    item_metadata.get('id').append(row[0])
item_metadata_df = pd.DataFrame.from_dict(item_metadata)


def build_subgraph_from_ratings(ratings: pd.DataFrame) -> Graph:
    sub_graph = Graph()

    for idx, row in context_metadata_df.iterrows():
        cid = row['id']
        sub_graph += graph.triples((cid, None, None))
        sub_graph += graph.triples((None, None, cid))

    for idx, row in user_metadata_df.iterrows():
        uid = row['id']
        sub_graph += graph.triples((uid, DASHB.memberOf, None))

    for idx, row in item_metadata_df.iterrows():
        iid = row['id']
        sub_graph += graph.triples((iid, None, None))
        sub_graph += graph.triples((None, None, iid))

    for idx, row in ratings.iterrows():
        uid = row['user']
        iid = row['item']
        cid = row['context']

        sub_graph += graph.triples((uid, None, None))
        sub_graph += graph.triples((None, None, uid))
        sub_graph += graph.triples((iid, None, None))
        sub_graph += graph.triples((None, None, iid))

        sub_graph += graph.triples((cid, None, None))
        sub_graph += graph.triples((None, None, cid))

    return sub_graph

In [12]:
class KFoldCrossValidation():

    def __init__(self, recommender: Recommender, metrics: [], k=5):
        """
        :param project_id: ID of the project (to load the correct data).
        :param recommender: Recommender
        :param metrics: List of Metrics
        :param k: Number of folds
        """
        self.recommender = recommender
        self.metrics = metrics
        self.k = k

    def evaluate(self, ratings, **kwargs):
        kf = KFold(n_splits=self.k, shuffle=True)

        n_fold = 0

        result = {'folds': [], 'result': {}}
        for train_idx, test_idx in kf.split(ratings):
            train = ratings.iloc[train_idx]
            test = ratings.iloc[test_idx]

            graph = build_subgraph_from_ratings(train)
            self.recommender.set_graph(graph)

            fold_scores = {}

            for uid in tqdm(test['user'].unique()):
                df_user = test.loc[test['user'] == uid]

                for cid in df_user['context']:
                    print(uid, cid)
                    predictions = self.recommender.predict([uid], [cid], **kwargs)
                    recommendations = [r['itemId'] for r in predictions]

                    truth = []
                    t_user = test.loc[test['user'] == uid]
                    for idx, row in t_user.iterrows():
                        if row['context'] == cid:
                            truth.append(str(row['item']))
                    # truth = list(test.loc[test['user'] == uid].loc[test['context'] == cid]['item'])
                    print(recommendations, truth)

                    for metric in self.metrics:
                        if str(metric) not in fold_scores.keys():
                            fold_scores[str(metric)] = []
                        score = metric.calculate(recommendations, truth)
                        fold_scores[str(metric)].append(score)

            result_for_fold = {}
            for metric, scores in fold_scores.items():
                avg = sum(scores) / len(scores)
                result_for_fold[metric] = avg

                if metric not in result['result'].keys():
                    result['result'][metric] = []
                result['result'][metric].append(avg)
            result['folds'].append(result_for_fold)

            n_fold += 1

        final_results = {}
        for metric_type, score in result['result'].items():
            final_results[metric_type] = sum(score) / len(score)

        result['result'] = final_results
        return result

In [13]:
metric_factory = MetricFactory()

metrics = ['f1@1', 'ndcg@1', 'ndcg@3']
parsed_metrics = []
for metric in metrics:
    m_split = metric.split('@')
    m_type = m_split[0]
    n = int(m_split[1]) if len(m_split) >= 2 else None

    metric_ = metric_factory.get(MetricType.reverse_lookup(m_type), n)
    parsed_metrics.append(metric_)

In [14]:
recommender = PPR(graph)
recommender.set_personalization(0.7, 0.3)

evaluator = KFoldCrossValidation(recommender, metrics=parsed_metrics, k=5)
result = evaluator.evaluate(ratings_df)

for fold in result['folds']:
    print(fold)
print(result['result'])

  0%|                                                                                            | 0/4 [00:00<?, ?it/s]

https://dynamicdashboard.ilabt.imec.be/users/7 http://example.com/tx/patients/zplus_2
['https://webthing.protego.dynamicdashboard.ilabt.imec.be/things/zplus_6.60%253A77%253A71%253A7D%253A93%253AD7%252Fservice0009/properties/org.dyamand.types.health.GlucoseLevel', 'https://webthing.protego.dynamicdashboard.ilabt.imec.be/things/zplus_6.lifestyle/properties/enriched-call', 'https://webthing.protego.dynamicdashboard.ilabt.imec.be/things/zplus_6.B0%253A91%253A22%253AFB%253AD0%253A78%252Fservice0009/properties/org.dyamand.types.health.GlucoseLevel', 'https://webthing.protego.dynamicdashboard.ilabt.imec.be/things/zplus_6.00%253A1C%253A05%253AFE%253A44%253A6B%252Fservice0029/properties/org.dyamand.types.health.SpO2', 'https://webthing.protego.dynamicdashboard.ilabt.imec.be/things/zplus_6.00%253A1C%253A05%253AFF%253AA9%253A4E%252Fservice0029/properties/org.dyamand.types.health.SpO2', 'https://webthing.protego.dynamicdashboard.ilabt.imec.be/things/zplus_6.F8%253A33%253A31%253A46%253A7F.D0%252Fse

 25%|█████████████████████                                                               | 1/4 [00:00<00:02,  1.42it/s]

['https://webthing.protego.dynamicdashboard.ilabt.imec.be/things/zplus_6.60%253A77%253A71%253A7D%253A93%253AD7%252Fservice0009/properties/org.dyamand.types.health.GlucoseLevel', 'https://webthing.protego.dynamicdashboard.ilabt.imec.be/things/zplus_6.lifestyle/properties/enriched-call', 'https://webthing.protego.dynamicdashboard.ilabt.imec.be/things/zplus_6.B0%253A91%253A22%253AFB%253AD0%253A78%252Fservice0009/properties/org.dyamand.types.health.GlucoseLevel', 'https://webthing.protego.dynamicdashboard.ilabt.imec.be/things/zplus_6.00%253A1C%253A05%253AFE%253A44%253A6B%252Fservice0029/properties/org.dyamand.types.health.SpO2', 'https://webthing.protego.dynamicdashboard.ilabt.imec.be/things/zplus_6.00%253A1C%253A05%253AFF%253AA9%253A4E%252Fservice0029/properties/org.dyamand.types.health.SpO2', 'https://webthing.protego.dynamicdashboard.ilabt.imec.be/things/zplus_6.F8%253A33%253A31%253A46%253A7F.D0%252Fservice0010/properties/org.dyamand.types.health.BodyTemperature', 'https://webthing.prot

 75%|███████████████████████████████████████████████████████████████                     | 3/4 [00:00<00:00,  3.92it/s]

...

['https://webthing.protego.dynamicdashboard.ilabt.imec.be/things/zplus_6.60%253A77%253A71%253A7D%253A93%253AD7%252Fservice0009/properties/org.dyamand.types.health.GlucoseLevel', 'https://webthing.protego.dynamicdashboard.ilabt.imec.be/things/zplus_6.lifestyle/properties/enriched-call', 'https://webthing.protego.dynamicdashboard.ilabt.imec.be/things/zplus_6.B0%253A91%253A22%253AFB%253AD0%253A78%252Fservice0009/properties/org.dyamand.types.health.GlucoseLevel', 'https://webthing.protego.dynamicdashboard.ilabt.imec.be/things/zplus_6.00%253A1C%253A05%253AFE%253A44%253A6B%252Fservice0029/properties/org.dyamand.types.health.SpO2', 'https://webthing.protego.dynamicdashboard.ilabt.imec.be/things/zplus_6.00%253A1C%253A05%253AFF%253AA9%253A4E%252Fservice0029/properties/org.dyamand.types.health.SpO2', 'https://webthing.protego.dynamicdashboard.ilabt.imec.be/things/zplus_6.F8%253A33%253A31%253A46%253A7F.D0%252Fservice0010/properties/org.dyamand.types.health.BodyTemperature', 'https://webthing.prot

 75%|███████████████████████████████████████████████████████████████                     | 3/4 [00:00<00:00,  3.87it/s]

https://dynamicdashboard.ilabt.imec.be/users/10 http://example.com/tx/patients/zplus_129
['https://webthing.protego.dynamicdashboard.ilabt.imec.be/things/zplus_6.60%253A77%253A71%253A7D%253A93%253AD7%252Fservice0009/properties/org.dyamand.types.health.GlucoseLevel', 'https://webthing.protego.dynamicdashboard.ilabt.imec.be/things/zplus_6.lifestyle/properties/enriched-call', 'https://webthing.protego.dynamicdashboard.ilabt.imec.be/things/zplus_6.B0%253A91%253A22%253AFB%253AD0%253A78%252Fservice0009/properties/org.dyamand.types.health.GlucoseLevel', 'https://webthing.protego.dynamicdashboard.ilabt.imec.be/things/zplus_6.00%253A1C%253A05%253AFE%253A44%253A6B%252Fservice0029/properties/org.dyamand.types.health.SpO2', 'https://webthing.protego.dynamicdashboard.ilabt.imec.be/things/zplus_6.00%253A1C%253A05%253AFF%253AA9%253A4E%252Fservice0029/properties/org.dyamand.types.health.SpO2', 'https://webthing.protego.dynamicdashboard.ilabt.imec.be/things/zplus_6.F8%253A33%253A31%253A46%253A7F.D0%252

100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:01<00:00,  3.15it/s]

['https://webthing.protego.dynamicdashboard.ilabt.imec.be/things/zplus_6.60%253A77%253A71%253A7D%253A93%253AD7%252Fservice0009/properties/org.dyamand.types.health.GlucoseLevel', 'https://webthing.protego.dynamicdashboard.ilabt.imec.be/things/zplus_6.lifestyle/properties/enriched-call', 'https://webthing.protego.dynamicdashboard.ilabt.imec.be/things/zplus_6.B0%253A91%253A22%253AFB%253AD0%253A78%252Fservice0009/properties/org.dyamand.types.health.GlucoseLevel', 'https://webthing.protego.dynamicdashboard.ilabt.imec.be/things/zplus_6.00%253A1C%253A05%253AFE%253A44%253A6B%252Fservice0029/properties/org.dyamand.types.health.SpO2', 'https://webthing.protego.dynamicdashboard.ilabt.imec.be/things/zplus_6.00%253A1C%253A05%253AFF%253AA9%253A4E%252Fservice0029/properties/org.dyamand.types.health.SpO2', 'https://webthing.protego.dynamicdashboard.ilabt.imec.be/things/zplus_6.F8%253A33%253A31%253A46%253A7F.D0%252Fservice0010/properties/org.dyamand.types.health.BodyTemperature', 'https://webthing.prot